#Exploration of Crown Height Mesurements

In [1]:
!pip install rasterio -q
!pip install geopandas -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 76.8 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np

import geopandas
import rasterio
import rasterio.plot
from rasterio.enums import Resampling

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
from ipywidgets import interact

from google.colab import drive
drive.mount('/content/drive/')

# dir_ = '/content/drive/MyDrive/ift6759_trees' # Anni
dir_ = '/content/drive/MyDrive/Mila/ift6759_trees' # Xavier



Mounted at /content/drive/


### Visualisation of CHM data

In [3]:
subset = 'test'
dir_in = f'{dir_}/data/processed/{subset}/RGBH/'
paths=[]
for subdir, dirs, files in os.walk(dir_in):
  for file in files:
      filepath = subdir + file
      if filepath.endswith(".tif"):
        paths.append(filepath)
def get_bboxes_from_df(df,rsFile):
  bboxes = []
  for index, row in df.iterrows():
    if row['rsFile'] == rsFile:
      class_, bboxe =  0, np.array([row['minx_pixel'],row['miny_pixel'],row['maxx_pixel']-row['minx_pixel'],row['maxy_pixel']-row['miny_pixel']])
      bboxes.append((class_,bboxe))
  return bboxes
subset_shp = os.path.join(dir_,f'data/interim/ITC/{subset}.shp') # subset shapefile path        
df = pd.read_csv(os.path.join(dir_,f'data/interim/ITC/{subset}_pixel_v3.csv'))

@ interact(id=widgets.IntSlider(min=0, max=len(paths)-1, step=1, value=0, description='Id #:'))
def display_RGBH(id):
    tiff = paths[id]
    dataset = rasterio.open(tiff)

  
    color = dataset.read([1,2,3])
    height = dataset.read(4)
 
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12, 4))
    fig.suptitle(os.path.basename(tiff))

    ax1.imshow(color.transpose(1,2,0)/255.0)
    ax1.set_title('RGB')
    bboxes = get_bboxes_from_df(df,os.path.basename(tiff))

    for class_, bboxe in bboxes:
      rect = patches.Rectangle((bboxe[0], bboxe[1]),bboxe[2], bboxe[3], linewidth=2, edgecolor=np.random.rand(3,), facecolor='none')
      ax1.add_patch(rect)
     

    ax2.imshow(height,cmap='Greys')
    ax2.set_title('Canopy height')

    # Show the figure
    plt.show()


interactive(children=(IntSlider(value=0, description='Id #:', max=8), Output()), _dom_classes=('widget-interac…

### Create 4D images that incorporate the CHM information

#### Get min and max from trainset to normalize images

In [ ]:
def min_max_chm(image):
   with rasterio.open(image) as src:
        band = src.read()
        return band.min(),band.max()

dir_in = f'{dir_}/data/interim/RemoteSensing/train/CHM/' 
mins = []
maxs = []   
for root, dirs, files in os.walk(dir_in, topdown=False):
    for name in files:
        if os.path.splitext(os.path.join(root, name))[1].lower() == ".tif":  
            min,max = min_max_chm(os.path.join(root, name))
            mins.append(min)
            maxs.append(max)

MAX = np.max(maxs)
MIN = np.min(mins)
print(MAX,MIN)

32.804 0.0


#### Define code to merge RGB and CHM

In [ ]:
def combine_images(image_3c, image_1c, out_path,channels_first_image=[1,2,3]):
    """
    This function takes a three-channel image and a one-channel image as inputs and returns a combined image with four channels.
    The function uses rasterio library to read and write images in various formats.
    Parameters:
        image_3c: a string representing the path to the three-channel image file
        image_1c: a string representing the path to the one-channel image file
    Returns:
        a string representing the path to the output image file with four channels
    """
    import rasterio
    # open the input images using rasterio
    with rasterio.open(image_3c) as src_3c, rasterio.open(image_1c) as src_1c:
        # check if the input images have compatible shapes and CRS
        if src_3c.shape != src_1c.shape or src_3c.crs != src_1c.crs:
            raise ValueError("The input images must have the same shape and CRS")
        # read the input images as numpy arrays
        array_3c = src_3c.read(channels_first_image)
        array_1c = src_1c.read()/MAX * 255
        
        # stack the arrays along the band axis to create a four-channel array
        array_4c = np.concatenate((array_3c, array_1c), axis=0)
        # create a profile for the output image using the metadata of the first input image
        profile = src_3c.profile
        # update the profile with the new count of bands and dtype
        profile.update(count=len(channels_first_image)+1, dtype=array_4c.dtype)
        
        
        # write the output image using rasterio
        with rasterio.open( out_path, "w+", **profile) as dst:
            dst.write(array_4c)
def scale_image(path, path_out, scale):
    # resample data to target shape
    dataset = rasterio.open(path)
    
    data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * scale),
            int(dataset.width * scale)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )
    profile = dataset.profile.copy()
    profile.update({"height": data.shape[-2],
                        "width": data.shape[-1],
                      "transform": transform})


    with rasterio.open(path_out, "w", **profile) as dataset:
        dataset.write(data)

def RGB_2_RGBA(RGB,A):
  RGB = cv2.imread(RGB)
  rgba = cv2.cvtColor(RGB, cv2.COLOR_RGB2RGBA)
  rgba[:, :, 3] = cv2.imread(A)

def create_RGBH(path_RGB,path_CHM,path_out,RGH=False):
  temp_path = 'temp.tif'
  scale_image(path_CHM,temp_path,10)
  if RGH:
    combine_images(path_RGB,temp_path,path_out,channels_first_image=[1,2])
  else:
    combine_images(path_RGB,temp_path,path_out)
  os.remove(temp_path)



def convert2RGBH(dir_in,dir_out,RGH=False):
  for subdir, dirs, files in os.walk(dir_in):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + file
       
        if filepath.endswith(".tif"):
          chm_path = os.path.dirname(os.path.dirname(filepath))
          chm_path = os.path.join(chm_path,'CHM',file)
         
          create_RGBH(filepath,chm_path,os.path.join(dir_out,file),RGH=RGH)




#### Merge RGB and CHM

In [ ]:
subsets = ['train','val','test']
for subset in subsets:      
  convert2RGBH(f'{dir_}/data/interim/RemoteSensing/{subset}/RGB/',f'{dir_}/data/processed/{subset}/RGH',RGH=True)

### Convert created images to PNGs

In [ ]:
subsets = ['train','val','test']
for subset in subsets:
  dir_out = f'{dir_}/data/processed/{subset}/RGH-png'
  dir_in = f'{dir_}/data/processed/{subset}/RGH'

  import os
  from PIL import Image
  import cv2
  os.makedirs(dir_out, exist_ok=True)
  for root, dirs, files in os.walk(dir_in, topdown=False):
      for name in files:
        path_in = os.path.join(root, name)
        if os.path.splitext(path_in)[1].lower() == ".tif":
            path_out = os.path.join(dir_out,os.path.splitext(name)[0]+'.png')
            img = cv2.imread(path_in, -1)
            cv2.imwrite(path_out,img)